In [48]:
import tweepy            
import pandas as pd      
import numpy as np       

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()

ACCESS_TOKEN = 'replace_with_yours'
ACCESS_SECRET = 'replace_with_yours'

CONSUMER_KEY = 'replace_with_yours'
CONSUMER_SECRET = 'replace_with_yours'

In [49]:
from credentials import *    # This will allow us to use the keys as variables

# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [50]:
# List of twitter username to crawl from 
USERS = ['cnbc', 'benzinga', 'stocktwits', 'breakoutstocks', 'WSJMarkets',
        'jimcramer', 'nytimesbusiness', 'IBDinvestors', 'WSJDealJournal',
        'LizAnnSonders', 'bySamRo', 'NorthmanTrader', 'CiovaccoCapital', 'JLyonsFundMGMT',
        'SconsetCapital', 'Amena__Bakr', 'IvanTheK', 'StockCats', 'TheStalwart',
        'Sassy_SPY', 'realDonaldTrump', 'jimcramer' ]

In [51]:
COUNT = 10 # number of tweets to crawl

# Create an extractor object:
extractor = twitter_setup()

# startDate = datetime.datetime(2018, 5, 1, 0, 0, 0)
# endDate =   datetime.datetime(2018, 9, 1, 0, 0, 0)


In [52]:
def get_user_tweet_df (user, count):
    
    tweets = extractor.user_timeline(screen_name=user, count=COUNT)
    data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
    data['len']  = np.array([len(tweet.text) for tweet in tweets])
    data['ID']   = np.array([tweet.id for tweet in tweets])
    data['Date'] = np.array([tweet.created_at for tweet in tweets])
    data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
    data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])
    data['User']    = np.array([tweet.user.name for tweet in tweets])
    
    return data
    

In [53]:
user_dfs = list(map(lambda x: get_user_tweet_df(x, COUNT), USERS))
tweets_aggreg = pd.concat(user_dfs)

In [55]:
tweets_aggreg.head()

,Tweets,len,ID,Date,Likes,RTs,User
0,Everyone could learn from China's tech policie...,98,1042287673505927168,2018-09-19 05:42:00,6,4,CNBC
1,Former US trade official: Deadlock with China ...,99,1042285116557586438,2018-09-19 05:31:51,5,6,CNBC
2,"Amazon is worth a lot of money. To be exact, i...",119,1042284652080259072,2018-09-19 05:30:00,10,9,CNBC
3,Coca-Cola exec has business advice for Trump a...,103,1042282753868357633,2018-09-19 05:22:27,16,12,CNBC
4,"European markets look to open higher, brushing...",88,1042281907030032384,2018-09-19 05:19:05,3,6,CNBC


In [57]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [58]:
# We create a column with the result of the analysis:
tweets_aggreg['SA'] = np.array([analize_sentiment(tweet) for tweet in tweets_aggreg['Tweets'] ])

# We display the updated dataframe with the new column:
display(tweets_aggreg.head(10))

,Tweets,len,ID,Date,Likes,RTs,User,SA
0,Everyone could learn from China's tech policie...,98,1042287673505927168,2018-09-19 05:42:00,6,4,CNBC,1
1,Former US trade official: Deadlock with China ...,99,1042285116557586438,2018-09-19 05:31:51,5,6,CNBC,0
2,"Amazon is worth a lot of money. To be exact, i...",119,1042284652080259072,2018-09-19 05:30:00,10,9,CNBC,1
3,Coca-Cola exec has business advice for Trump a...,103,1042282753868357633,2018-09-19 05:22:27,16,12,CNBC,0
4,"European markets look to open higher, brushing...",88,1042281907030032384,2018-09-19 05:19:05,3,6,CNBC,1
5,Technology used by NASA could end food shortag...,102,1042278461111513088,2018-09-19 05:05:24,41,23,CNBC,0
6,Jack Ma says U.S.-China trade frictions could ...,136,1042271265896255488,2018-09-19 04:36:48,45,21,CNBC,-1
7,"You've probably heard of ""football widows"" — w...",134,1042270148659478528,2018-09-19 04:32:22,17,7,CNBC,0
8,Apple announced a more affordable version of t...,140,1042269300185169920,2018-09-19 04:29:00,19,10,CNBC,1
9,JP Morgan Chase: Emerging markets selloff is o...,114,1042268193484677120,2018-09-19 04:24:36,26,18,CNBC,0


In [62]:
tweets_aggreg['Tweets']

0    Everyone could learn from China's tech policie...
1    Former US trade official: Deadlock with China ...
2    Amazon is worth a lot of money. To be exact, i...
3    Coca-Cola exec has business advice for Trump a...
4    European markets look to open higher, brushing...
5    Technology used by NASA could end food shortag...
6    Jack Ma says U.S.-China trade frictions could ...
7    You've probably heard of "football widows" — w...
8    Apple announced a more affordable version of t...
9    JP Morgan Chase: Emerging markets selloff is o...
0    Wherein .@Robinhood sheds some light on its un...
1    "Success usually comes to those who are too bu...
2    Apple's Tim Cook describes how the iPhone esca...
3    The Companies That Could Benefit From An Auror...
4    Fun facts on hot pot stock Tilray $TLRY:\n- Sh...
5    In an SEC filing early this morning, Eventbrit...
6    Tesla full statement on DoJ probe $TSLA: https...
7    Tesla shares ⬇️ 5% after DoJ reportedly workin...
8    Tesla

In [80]:
import datetime
now = datetime.datetime.now().now().strftime("%Y-%m-%d")

In [81]:
filename = 'tweets' + now + '.txt'
f = open(filename, "w")

In [82]:
for line in pd.Series(tweets_aggreg['Tweets']):
    f.write(line)

In [83]:
f.close()

In [85]:
for line in pd.Series(tweets_aggreg['Tweets']):
    print(line)

Everyone could learn from China's tech policies, World Economic Forum says https://t.co/JJV29NWjwO
Former US trade official: Deadlock with China will continue 'for some time' https://t.co/ODHTLGyoht
Amazon is worth a lot of money. To be exact, it's worth more than 21 major retailers combined.… https://t.co/UW4uKb3A9k
Coca-Cola exec has business advice for Trump and Xi: 'Exchange WeChat accounts' https://t.co/AhYxpuXFFk
European markets look to open higher, brushing off trade dispute https://t.co/KmAmvwtAIu
Technology used by NASA could end food shortages and save trillions of dollars https://t.co/ytL9NIE8df
Jack Ma says U.S.-China trade frictions could last for 2 decades and would be "a mess" for all parties involved. https://t.co/pwmKqGW7jH
You've probably heard of "football widows" — well now it seems there are "Fortnite" divorcees. via @CNBCMakeIt https://t.co/Qu1bVwZ3Io
Apple announced a more affordable version of the iPhone X called the iPhone XR. The price will start at $749 and 